In [1]:
import sqlite3
import pandas as pd
import numpy as np

## Convert csv to SQL database

In [2]:
df = pd.read_csv('twitter_archive_master.csv')

In [4]:
conn = sqlite3.connect('database.db')
c = conn.cursor()

In [32]:
df.to_sql("dog_tweets", conn, index=False)

In [46]:
df.head()

,tweet_id,timestamp,text,expanded_urls,name,doggo,floofer,pupper,puppo,retweet_count,favorite_count,jpg_url,pred,conf,rating,dog_stage
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,False,False,False,False,8853,39467,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,NaN,NaN,13/10,doggo
1,892177421306343426,2017-08-01 00:17:27,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,False,False,False,False,6514,33819,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,Chihuahua,0.32,13/10,doggo
2,891815181378084864,2017-07-31 00:18:03,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,False,False,False,False,4328,25461,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,Chihuahua,0.72,12/10,doggo
3,891689557279858688,2017-07-30 15:58:51,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,False,False,False,False,8964,42908,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,Labrador_retriever,0.17,13/10,doggo
4,891327558926688256,2017-07-29 16:00:24,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,Franklin,False,False,False,False,9774,41048,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,basset,0.56,12/10,doggo


In [33]:
dog_keywords = df[['doggo','puppo','pupper','floofer']]
dog_stage = dog_keywords.idxmax(1).to_frame('dog_stage')

In [28]:
dog_stage.dog_stage.value_counts()

doggo      1761
pupper      203
puppo        22
floofer       7
Name: dog_stage, dtype: int64

In [29]:
df.doggo.replace('None', '', inplace=True)
df.floofer.replace('None', '', inplace=True)
df.pupper.replace('None', '', inplace=True)
df.puppo.replace('None', '', inplace=True)

# merge into column
df['dog_stage'] = dog_stage.dog_stage

In [49]:
c.execute("""SELECT timestamp
FROM dog_tweets;""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head()

,timestamp
0,2017-08-01 16:23:56
1,2017-08-01 00:17:27
2,2017-07-31 00:18:03
3,2017-07-30 15:58:51
4,2017-07-29 16:00:24


## Agreggate mean retweet count per dog_stage

In [45]:
c.execute("""SELECT dog_stage,
AVG(retweet_count) AS avg_retweet_count
FROM dog_tweets
GROUP BY dog_stage
ORDER BY avg_retweet_count DESC;""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head()

,dog_stage,avg_retweet_count
0,puppo,6473.954545
1,floofer,4968.714286
2,doggo,2758.793867
3,pupper,2363.581281


## Use Subquery/ temporary table to aggregate total retweets per predicted breed, in descending order

In [62]:
c.execute(""" 
SELECT pred,
SUM(retweet_count) AS total_retweets
FROM (SELECT pred,
retweet_count
FROM dog_tweets) sub
GROUP BY pred
ORDER BY total_retweets DESC;""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head(25)

,pred,total_retweets
0,None,767245
1,golden_retriever,585873
2,Labrador_retriever,407573
3,Pembroke,292052
4,Chihuahua,251807
5,Samoyed,203446
6,French_bulldog,156230
7,cocker_spaniel,145743
8,chow,131025
9,pug,118682


## Use Window function to create running favorite_count minimum by month, in chronological order

In [61]:
c.execute(""" 
SELECT timestamp,
favorite_count,
MIN(favorite_count) OVER 
(PARTITION BY strftime('%M-%Y', timestamp) 
ORDER BY timestamp) AS monthly_min
FROM dog_tweets;""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head(25)

,timestamp,favorite_count,monthly_min
0,2015-11-17 02:00:15,170,170
1,2015-11-21 04:00:28,222,170
2,2015-11-26 01:00:28,375,170
3,2015-11-28 02:00:17,1415,170
4,2015-12-01 17:00:19,1534,170
5,2015-12-04 22:00:08,944,170
6,2015-12-05 04:00:04,1039,170
7,2015-12-18 04:00:46,3689,170
8,2015-12-24 16:00:30,18278,170
9,2015-12-24 17:00:27,2818,170


## Use Window function to create moving favorite_count average over last 90 tweets, in chronological order

In [67]:
c.execute(""" 
SELECT timestamp,
favorite_count,
AVG(favorite_count) OVER (ORDER BY timestamp ASC ROWS 90 PRECEDING) AS favorite_count_moving_average
FROM dog_tweets;""")
data = pd.DataFrame(c.fetchall())
data.columns = [x[0] for x in c.description]
data.head(25)

,timestamp,favorite_count,favorite_count_moving_average
0,2015-11-15 22:32:08,2535,2535.000000
1,2015-11-15 23:05:30,132,1333.500000
2,2015-11-15 23:21:54,128,931.666667
3,2015-11-16 00:04:52,311,776.500000
4,2015-11-16 00:24:50,111,643.400000
5,2015-11-16 00:30:50,136,558.833333
6,2015-11-16 00:35:11,1253,658.000000
7,2015-11-16 00:49:46,448,631.750000
8,2015-11-16 00:55:59,304,595.333333
9,2015-11-16 01:01:59,115,547.300000
